# Анализ рынка квартир в Санкт-Петербурге

В нашем распоряжении данные сервиса Яндекс Недвижимость — архив объявлений за несколько лет о продаже квартир в Санкт-Петербурге и соседних населённых пунктах. В рамках данного проекта наша задача — выполнить предобработку данных и изучить их, чтобы найти интересные особенности и зависимости, которые существуют на рынке недвижимости.<br>
О каждой квартире в базе содержится два типа данных: добавленные пользователем и картографические. Например, к первому типу относятся площадь квартиры, её этаж и количество балконов, ко второму — расстояния до центра города, аэропорта и ближайшего парка.

## 1. Загрузка и обзор датафрейма

Импортируем библиотеку pandas для работы с набором данных, библиотеку matplotlib для отображения графиков,библиотеку plotly для более удобной визуализации и библиотеку missingno для простой визуализации пропусков:

In [ ]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
#!pip install missingno
import missingno as msno

Считаем в переменную flats наши данные и посмотрим на первые 5 строк таблицы:

In [ ]:
flats = pd.read_csv('real_estate_data.csv', sep='\t')
flats.head()

Узнаем теперь, с каким количеством данных имеем дело:

In [ ]:
flats.shape

Итак, в нашей таблице 23699 строк и 22 столбца, получим о них дополнительную информацию:

In [ ]:
flats.info()

Как можно видеть, данный набор данных содержит весьма большое количество пропусков. С ними разберемся в следующей части исследования, а сейчас построим общую гистограмму для нашей таблицы:

In [ ]:
flats.hist(figsize=(20, 20), color='indigo')
plt.show()

Перед тем, как делать по гистограммам какие-то выводы, займемся предобработкой данных.

## 2. Предобработка данных

### 1. Работа с пропусками

Сперва займемся обработкой пропусков. Визуально оценим их количество в каждом столбце:

In [ ]:
msno.matrix(flats, labels=True)

Как видим, многие столбцы содержат пропуски, причем в некоторых (например, в 'is_apartment', 'parks_nearest', 'ponds_nearest') их количество весьма ощутимо.<br>
Попробуем, насколько это возможно, найти причину наличия пропусков и отредактируем в соответствии с ней наш датафрейм. В тех столбцах, где способ заполнения пропусков найти не удастся - оставим все как есть.

В столбцах 'ceiling_height', 'floors_total', 'living_area', 'kitchen_area', 'locality_name', 'airports_nearest', 'cityCenters_nearest', 'parks_nearest', 'ponds_nearest' и 'days_exposition' выявить какую-то логику, позволяющую заменить пропуски какими-то значениями, сложно. Нереалистичной кажется гипотеза о том, что пропуск на самом деле говорит о нулевом значении (в доме не может быть 0 этажей, до ближайшего водоема не может быть 0 метров, название населенного пункта не может быть пустым). Попробуем выдвинуть гипотезы о причинах наличия этих пропусков ниже.

Данные в столбцы 'ceiling_height', 'floors_total', 'living_area', 'kitchen_area', 'locality_name', судя по всему, заносятся пользователями. Тогда можно предположить, что в случае со столбцами 'ceiling_height', 'living_area' и 'kitchen_area' пользователь не имел представления о значении конкретной характеристики, не имел возможности или желания их измерить.<br>
То же сложно сказать о данных в столбцах 'floors_total' и 'locality_name' - трудно представить, что хозяин квартиры не знает, сколько этажей в его доме, и в каком населенном пункте он живет. Скорее всего, тут мы имеем дело с ошибкой в сборе данных, что может подтверждать и очень малое количество таких пропусков - 86 и 49 соответственно.

Теперь обратимся к пропускам в столбцах 'airports_nearest', 'cityCenters_nearest', 'parks_nearest' и 'ponds_nearest'. Эти данные, как это представляется, должны определяться автоматически. Сперва разберемся со столбцами 'parks_nearest' и 'ponds_nearest'. Как можно видеть в коде ниже, ситуаций, в которых при отсутствии парка или пруда в радиусе 3000 метров значением столбцов 'parks_nearest' и 'ponds_nearest' является NaN почти столько же (в случае с прудами - в точности столько же), сколько и просто ситуаций, в которых в данных столбцах стоит NaN. Тогда предположим, что когда в необходимом радиусе нет указанных объектов, расстоянию до ближайшего из них автоматически не присваивается никакого значения. Исправить данный пропуск, наверное, можно, подставив вместо него любое значение больше 3000, к примеру, 3001. Однако, это может создать перекос при анализе данных, поэтому никак его трогать не будем.

In [ ]:
flats[flats['parks_around3000'] == 0].shape[0], flats[(flats['parks_around3000'] == 0) & (flats['parks_nearest'].isna() == True)].shape[0]

In [ ]:
flats[flats['ponds_around3000'] == 0].shape[0], flats[(flats['ponds_around3000'] == 0) & (flats['ponds_nearest'].isna() == True)].shape[0]

Рассмотрим столбцы 'airports_nearest' и 'cityCenters_nearest'. Условно можно объяснить пропуск в этих данных так - "Аэропорта рядом нет, центр города - далеко", но не вполне ясно, в каком радиусе ищутся эти расстояния, ведь любой точке на карте можно сопоставить ближайший аэропорт или вычислить ее расстояние до центра определенного города. Процедура поиска указанных расстояний нам неочевидна, поэтому точно так же ничего с этими пропусками делать не будем.

Остался последний столбец из тех, в которых неочевидна возможность пропуски исправить - 'days_exposition'. Рассмотрим предположение о том, что на месте пропуска должен стоять 0.<br>
В остальных данных случаев, когда значение в этом столбце было бы нулевым, - нет, что следует из кода:

In [ ]:
flats[flats['days_exposition'] == 0].shape[0]

О чем могло бы говорить нулевое значение в данном столбце? Возможно, о снятии объявления раньше, чем за 1 день. Для понимания адекватности этой гипотезы найдем, сколько объявлений сняли за прилегающий период.<br>
Обратимся к коду ниже - объявлений, снятых с сервиса в интервале от 1 до 10 дней - 1257, что в 2,5 раза меньше, чем объявлений, снятых менее чем за 1 день (за количество которых принимаем по гипотезе количество пропусков - 3181). Представляется маловероятным, что количество снятых меньше чем за сутки объвлений настолько больше, чем суммарное количество объявлений, снятых за следующие 10 дней. Видимо, гипотеза о нулевом значении в данном случае не соответствует реальности. В качестве других причин можно опять предложить ошибку в сборе данных и автоматическом присваивании значений - снова оставляем пропуск нетронутым.

In [ ]:
flats[(flats['days_exposition'] >= 1) & (flats['days_exposition'] <= 10)].shape[0]

Пропуски же в столбцах 'is_apartment', 'balcony', 'parks_around3000' и 'parks_around3000' вполне объяснимы, во всех случаях пропуск выражает ответ "нет" на соответствующий вопрос: "Является ли квартира аппартаментами?", "Есть ли в квартире балкон?", "Есть ли парки или водоемы в радиусе 3000 метров от дома"? Тогда заполним пропуски соответствующими значениями:

In [ ]:
values = {'is_apartment': False, 'balcony': 0, 'parks_around3000': 0, 'parks_around3000': 0}
flats = flats.fillna(value=values)

Удостоверимся, что в выбранных столбцах больше нет пропусков:

In [ ]:
flats[['is_apartment', 'balcony', 'parks_around3000', 'parks_around3000']].isna().sum()

### 2. Преобразование типов

Вспомним, каким типом данных записывается каждый из признаков (столбцов):

In [ ]:
flats.info()

Получим случайный срез данных, попробуем оценить, какие столбцы могут быть представлены иным, более подходящим им типом. Для этого потребуем показывать нам все столбцы датафрейма:

In [ ]:
pd.set_option('display.max_columns', None)
flats.sample(5)

Складывается впечатление, что данные в столбцах 'last_price', 'total_area', 'floors_total', 'balcony', 'airports_nearest', 'cityCenters_nearest', 'parks_around3000', 'parks_nearest', 'ponds_around3000', 'ponds_nearest' и 'days_exposition' представлены в виде целых чисел, что позволяет в целях экономии памяти изменить их тип на int32.<br>
Однако, как это видно выше, лишь колонки 'last_price', 'total_area', 'balcony' и 'parks_around3000' не содержат в себе пропусков, что в этой ситуации важно. Дело в том, что pandas хранит числовые колонки с NaN как колонки типа float64, и не поддерживает возможность их перевода в другие типы.<br>
В связи с этим ограничением, переведем в целочисленный тип лишь столбцы без пропусков:

In [ ]:
cols_to_convert = ['last_price', 'total_area', 'balcony', 'parks_around3000']
for col in cols_to_convert:
    flats[col] = flats[col].astype('int32')

Удостоверимся, что соответствующие колонки приняли тип int32:

In [ ]:
flats[['last_price', 'total_area', 'balcony', 'parks_around3000']].info()

Вновь посмотрим на случайный срез данных:

In [ ]:
flats.sample(5)

Как видно, столбец 'first_day_exposition' для удобства работы с датами можем перевести в формат datetime64, заметив, к тому же, что необязательно включать  в этот столбец время, так как во всех записях оно равняется 00:00:00.

In [ ]:
flats['first_day_exposition'] = pd.to_datetime(flats['first_day_exposition'], format = '%Y-%m-%d')

Убедимся, что столбец перезаписан и теперь имеет необходимый нам тип:

In [ ]:
flats['first_day_exposition'].dtype.name

### 3. Устранение дубликатов

Рассмотрим уникальные значения в столбце, содержащем названия населенных пунктов, и устраним неявные дубликаты:

In [ ]:
flats['locality_name'].unique()

Объем данных очень велик, посмотрим, с каким количеством записей имеем дело:

In [ ]:
len(flats['locality_name'].unique())

Постараемся как-то упростить свою работу с неявными дубликатами, так как искать их вручную - явно не самая практичная идея.

Избавимся от характеристики населенного пункта, так, вместо 'поселок Володарское' будем писать просто 'Володарское' и т.п. Позволяет нам это сделать отсутсвие необходимости учитывать тип жилого образования в исследовании.

In [ ]:
locality_types = (['поселок', 'посёлок', 'городского типа', 'городской', 'коттеджный', 'станции', 
                   'при железнодорожной', 'садовое товарищество', 'садоводческое некоммерческое товарищество', 
                   'деревня', 'село'])

for i in range(len(flats['locality_name'])):
    if pd.isna(flats.loc[i, 'locality_name']) != True:
        for locality_type in locality_types:
            if locality_type in flats.loc[i, 'locality_name']:
                flats.loc[i, 'locality_name'] = flats.loc[i, 'locality_name'].replace(locality_type, '', 1)
                flats.loc[i, 'locality_name'] = flats.loc[i, 'locality_name'].lstrip()#Удалим пропуски в начале строки

В ходе работы вышенаписанного кода мы изменили название населенного пункта 'поселок Жилпосёлок' на 'Жил', исправим ситуацию:

In [ ]:
flats.loc[flats['locality_name'] == 'Жил', 'locality_name'] = 'Жилпосёлок'

Посмотрим, насколько сократился список уникальных названий:

In [ ]:
len(flats['locality_name'].unique())

По итогу проведенной работы - сократили количество уникальных названий населенных пунктов на 59 вариативных именований.

### 4. Обработка аномальных значений

Ознакомимся с описанием числовых данных, перед чем уточним формат отображаемых данных. Ради простоты анализа не будем рассматривать никакие процентили, кроме медианы:

In [ ]:
pd.options.display.float_format = '{:.2f}'.format
flats.describe(percentiles=[.5])

Необычные максимум и минимум в столбце 'last_price', рассмотрим соответствующие им строки:

In [ ]:
flats.query('last_price == last_price.min() or last_price == last_price.max()')

Очень подозрительной кажется минимальная цена в столбце - стометровую квартиру вряд ли продали за 12 тысяч рублей. Посмотрим, насколько она отличается от ближайших по цене вариантов:

In [ ]:
flats['last_price'].sort_values().head(10)

In [ ]:
(flats
    .query('last_price <= 470000')
    .sort_values(by='last_price')
    .head()
)

Вполне ожидаемо, 4 ближайшие по цене квартиры обладают небольшой площадью и расположены в небольших населенных пунктах, что позволяет считать их цену адекватной рынку. Минимальная же по цене квартира парадоксальным образом расположена в Санк-Петербурге, имеет большой метраж, отличаясь от ближайшей по стоимости квартиры в 35 (!) раз.

Похоже на то, что цену квартиры указали в тысячах, если так, то эту аномалию легко исправить. Проверим, есть ли аналогичные по метражу, количеству комнат и локации квартиры, находящиеся в предполагаемом ценовом диапазоне:

In [ ]:
(flats
    .query('last_price >= 11000000 and last_price <= 13000000 \
            and total_area >= 100 and total_area <= 120 \
            and locality_name == "Санкт-Петербург" \
            and rooms <= 2')
    .sort_values(by='last_price')
    .head()
)

Как видим, цена для данного сегмента квартир реалистичная, тогда отредактируем стоимость нашей аномально дешевой квартиры:

In [ ]:
flats.loc[flats['last_price'] == flats['last_price'].min(), 'last_price'] *= 1000

Теперь посмотрим, как обстоят дела с соседями по цене у самой дорогой квартиры:

In [ ]:
flats['last_price'].sort_values(ascending=False).head()

In [ ]:
(flats
    .query('last_price >= 190870000')
    .sort_values(by='last_price', ascending=False)
    .head()
)

От ближайшего соседа по цене наша квартира отличается почти в 2 раза, тем не менее, допустим, что цена указана верно: квартира, например, может располагаться в элитном жилом комплексе, идти вместе с личным машиноместом, иметь дорогой ремонт или индивидуальный дизайн.

Теперь обратим внимание на крайние значения в столбце 'ceiling_height', начнем с минимального:

In [ ]:
flats['ceiling_height'].sort_values().head()

Высота потолков меньше двух метров кажется очень подозрительной - удалим такие квартиры из датафрейма:

In [ ]:
flats = flats.query('ceiling_height >= 2 or ceiling_height.isna()')

Теперь посмотрим, каково положение дел с аномально большими значениями:

In [ ]:
flats['ceiling_height'].sort_values(ascending = False).head()

Представляется, что квартиру со 100-метровыми потолками стоит удалить из данных, а квартиры, превышающие 20-метровую высоту, отредактировать, разделив высоту на 10:

In [ ]:
flats.loc[flats['ceiling_height'] >= 20, 'ceiling_height'] /= 10

In [ ]:
flats = flats.query('ceiling_height != ceiling_height.max() or ceiling_height.isna()')

Взглянем теперь на лидеров по высоте потолков:

In [ ]:
flats.sort_values(by='ceiling_height', ascending=False).head()

Квартиры с высотой потолков 8 и больше метров кажутся подозрительными - они сравнительно малы по площади, сложно преставить, что на рынке существуют, к примеру, однокомнатные квартиры площадью 30 метров с потолками высотой 10 метров. Ниже восьмиметрового порога квартиры выглядят более реалистично - это большие квартиры, для которых высокие потолки более характерны. Очистим наши данные от квартир выше 8 метров:

In [ ]:
flats = flats.query('ceiling_height < 8 or ceiling_height.isna()')

Обратим теперь внимание на максимальные значения в колонке 'floors_total':

In [ ]:
flats['floors_total'].sort_values(ascending=False).head()

Насколько известно автору, в Петербурге и Ленинградской области нет жилых домой в 60 этажей, так же как и в 52 этажа. Удалим строки, содержащие 2 самых больших значения: 

In [ ]:
flats = flats.query('floors_total < 52 or floors_total.isna()')

Заметим необычайно малое минимальное значение в столбце 'living_area', разберемся, что там происходит:

In [ ]:
flats['living_area'].sort_values().head(15)

Автор находит нереалистично малой площадь жилого пространства, составляющую менее 9 метров, в связи с чем считает подозрительными соответствующие объявления. Удалим такие строки.<br>
Возможно, эта осторожность излишня, но информация о жилой площади еще пригодится нам в исследовании, будем считать, что таким образом обходим риск получения недостоверных сведений в результате исследования.

In [ ]:
flats = flats.query('living_area >= 9 or living_area.isna()')

Кажется не вполне соответствующим реальности количество объявлений с 4 и 5 балконами:

In [ ]:
len(flats.query('balcony == 4'))#Количество объявлений с 4 балконами

In [ ]:
len(flats.query('balcony == 5'))#Количество объявлений с 5 балконами

Кроме того, медианная площадь таких квартир невелика:

In [ ]:
flats.query('balcony == 4')['total_area'].median()#Медианная площадь квартир с 4 балконами

In [ ]:
flats.query('balcony == 5')['total_area'].median()#Медианная площадь квартир с 5 балконами

Но, так как в следующих этапах нашего исследования эти характеристики квартир нам не понадобятся, не будем предпринимать никаких шагов.

Теперь удалим из столбца 'airports_nearest' строку со значением 0 - квартира не может находиться в аэропорте

In [ ]:
flats = flats.query('airports_nearest != 0 or airports_nearest.isna()')

Обратимся к описанию нечисловых признаков:

In [ ]:
flats.describe(include=['object', 'bool'])

Среди них подозрительных данных не наблюдается.

## 3. Пополнение датафрейма новыми столбцами

Добавим в таблицу столбец 'price_per_meter', содержащий информацию о стоимости квадратного метра каждой квартиры:

In [ ]:
flats['price_per_meter'] = flats['last_price'] / flats['total_area']  

Теперь добавим столбец 'weekday_exposition', сообщающий информацию о том, в какой день недели была сделана публикация:

In [ ]:
flats['weekday_exposition'] = flats['first_day_exposition'].dt.weekday

Создадим также столбцы 'month_exposition' и 'year_exposition', хранящие информацию о месяце и годе публикации объявления соответственно:

In [ ]:
flats['month_exposition'] = flats['first_day_exposition'].dt.month
flats['year_exposition'] = flats['first_day_exposition'].dt.year

Добавим столбец 'floor_type', который разобьет этажи, на которых расположены квартиры, на типы "первый", "последний", "другой":

In [ ]:
def categorize_floor(floor, top_floor):
    if floor == 1:
        return 'первый'
    elif floor == top_floor:
        return 'последний'
    else:
        return 'другой'

In [ ]:
flats['floor_type'] = flats.apply(lambda x: categorize_floor(x.floor, x.floors_total), axis=1)

Наконец создадим столбец 'cityCenters_nearest_km', содержащий расстояние до центра города в километрах.

In [ ]:
flats['cityCenters_nearest_km'] = round(flats['cityCenters_nearest'] / 1000)

## 4. Исследовательский анализ данных

### 1. Изучение некоторых характеристик данных

С помощью построения гистограмм исследуем некоторые характеристики представленных в данных объектов (квартир). Для удобства оценки размаха данных под графиком будем выводить минимальноt и максимальное значения анализируемого столбца.

Начнем с такого параметра как 'total_area':

In [ ]:
fig = px.histogram(flats, x='total_area', title='Гистограмма общей площади', 
                   nbins=180, color_discrete_sequence=['indianred'])
fig.show()

In [ ]:
flats['total_area'].min(), flats['total_area'].max()

Как видно по графику, чаще всего площади квартир в объявлениях составляют от 30 до 44 метров, имея разброс от 12 до 900 метров. Распределение не является нормальным - налицо асимметрия с положительным коэффициентом (график скошен вправо). Также налицо и выбросы, находящиеся в правом конце хвоста - это квартиры с большой площадью, являющиеся редкостью на рынке.

Построим теперь гистограмму для столбца 'living_area':

In [ ]:
fig = px.histogram(flats, x='living_area', title='Гистограмма жилой площади', 
                   nbins=180, color_discrete_sequence=['indianred'])
fig.show()

In [ ]:
flats['living_area'].min(), flats['living_area'].max()

Как видим, график является мультимодальным, так как имеет три пика. Самые часто встречающиеся жилые площади в квартирах следующие - от 15 до 20 метров, от 30 до 35 метров и от 40 до 45 метров (интервалы указаны в порядке убывания частотности). По понятной причине размах так же, как и в случае с общей жилплощадью, весьма широк - от 9 до более чем 400  метров. В данном случае распределение так же не является нормальным - кроме мультимодальности имеем скос графика вправо. Выбросы на графике объясняются аналогичным предыдущему графику образом - ожидаемым образом, у больших квартир на рынке большие жилые пространства.

Теперь проанализируем последний столбец, содержащий информацию о площади - 'kitchen_area':

In [ ]:
fig = px.histogram(flats, x='kitchen_area', title='Гистограмма кухонной площади', 
                   nbins=110, color_discrete_sequence=['indianred'])
fig.show()

In [ ]:
flats['kitchen_area'].min(), flats['kitchen_area'].max()

Как видно на графике, самая часто встречающаяся площадь кухни - от 8 до 10 метров, а если брать более широкий интервал - от 6 до 12 метров. Ситуация с разбросом та же, что и на двух предыдущих графиках, характеризующих площади - минимальная площадь кухни составляет меньше полутора метров (1.3), максимальная - 112. Стоит заметить, что, в то время как разброс общей площади составляет 888 метров, а разброс жилой - 400.7, разброс кухонной площади равен 110.7 метрам, видим пропорциональное снижение (что неудивительно, так как жилая площадь и площадь кухни входят в общую площадь квартиры). Распределение напоминает нормальное, имеется небольшой скос графика вправо. Про выбросы было подробно сказано выше.

Рассмотрим теперь гистограмму по столбцу 'last_price', в котором располагается информация о цене объекта недвижимости:

In [ ]:
fig = px.histogram(flats, x='last_price', title='Гистограмма цены квартиры', 
                   color_discrete_sequence=['indianred'])
fig.show()

In [ ]:
flats['last_price'].min(), flats['last_price'].max()

Чтобы составить более детальное представление об основной части графика, ограничим размах значений по оси x: 

In [ ]:
fig.update_xaxes(range=[0,20000000])

Самый частотный интервал цен на квартиру - 3.5 - 4 миллиона, при чрезвычайно широком размахе в 762570000. При ограничении графика по оси абсцисс до значения в 20 миллионов становится видно, что распределение очень напоминает нормальное с небольшим положительным коэффициентом асимметрии. Выбросы на графике порождены наличием в датафрейме дорогих квартир, видимо, принадлежащих к сегменту элитной недвижимости или недвижимости бизнес-класса. 

Построим гистограмму, отображающую частотную встречаемость того или иного количества комнат в квартирах из нашего набора данных, для этого нам понадобится столбец 'rooms':

In [ ]:
fig = px.histogram(flats, x='rooms', title='Гистограмма количества комнат', 
                   color_discrete_sequence=['indianred'])
fig.show()

In [ ]:
flats['rooms'].min(), flats['rooms'].max()

Видим, что чаще всего появляются объявления о квартирах с 1 и 2 комнатами Наибольшее число комнат - 19, наименьшее - 0. Сразу скажем, что наличие квартир с нулевым числом комнат объясняется тем, что все эти квартиры либо обладают свободной планировкой, либо являются студиями, либо - аппартаментами, в чем легко убедиться:

In [ ]:
(len(flats.query('rooms == 0 and (is_apartment == True or studio == True or open_plan == True)')) == 
 len(flats.query('rooms == 0')))

График демонстрирует нормальное распределение с несколькими выбросами со стороны его правого хвоста. Здесь дело опять в редко встречающихся в данных больших по площади и количеству комнат квартирах. 

Обратимся теперь к столбцу 'ceiling_height':

In [ ]:
fig = px.histogram(flats, x='ceiling_height', title='Гистограмма высоты потолков', 
                   nbins=40, color_discrete_sequence=['indianred'])
fig.show()

In [ ]:
flats['ceiling_height'].min(), flats['ceiling_height'].max()

Среди имеющихся в таблице квартир превалируют те, высота потолков которых находится в интервале 2.4 - 2.8 метров, сами значения высоты начинаются с 2 метров и доходят до 6. Распределение асимметричное, имеет скос вправо, что вполне объяснимо - стандартная высота потолков квартиры начинается с 2-3 метров, квартиры же с более высокими потолками как правило являются более редкими и дорогими, что объясняет и выбросы на графе. 

Далее - построим гистограмму по столбцу 'floor', обозначающему этаж, на котором находится квартира:

In [ ]:
fig = px.histogram(flats, x='floor', title='Гистограмма этажа квартиры', 
                   nbins=40, color_discrete_sequence=['indianred'])
fig.show()

In [ ]:
flats['floor'].min(), flats['floor'].max()

Как видим, чаще всего в наш датасет попадают объявления о продаже квартир на первых трех этажах. Самое частое - на втором, затем - на третьем и, наконец, - на первом этаже. Самый высокий же этаж, на котором предлагается купить квартиру, - 33. Снова наблюдается большой правосторонний скос графика.

Продолжаем исследовать информацию относительно этажей, на этот раз рассмотрим данные из колонки 'floor_type':

In [ ]:
fig = px.histogram(flats, x='floor_type', title='Гистограмма типа этажа квартиры', 
                   nbins=40, color_discrete_sequence=['indianred'])
fig.show()

Любопытным образом, квартир на первом этаже продается почти столько же, сколько и на последнем - 2913 и 3333 соответственно. Больше же всего (17426), понятное дело, продается квартир на промежуточных этажах.

И, завершая анализ информации об этажах, с помощью столбца 'floors_total' проведем обзор данных об этажности домов, в которых можно купить квартиру:

In [ ]:
fig = px.histogram(flats, x='floors_total', title='Гистограмма этажности дома', 
                   nbins=40, color_discrete_sequence=['indianred'])
fig.show()

In [ ]:
flats['floors_total'].min(), flats['floors_total'].max()

В данных представлены дома этажностью от 1 до 36, более всего - дома пятиэтажные и девятиэтажные. Наблюдаем мультимодальный график с пиками в значениях 5, 9, 16, 12 и 25. Именно такие пиковые значения неудивительны - они отображают количество этажей в типовых домах, превалирующих в городской застройке. 

Дальше поработаем с расстояниями и начнем с расстояния до центра города в метрах, информация о котором содержится в колонке 'cityCenters_nearest'

In [ ]:
fig = px.histogram(flats, x='cityCenters_nearest', title='Гистограмма расстояния до центра города', 
                   color_discrete_sequence=['indianred'])
fig.show()

In [ ]:
flats['cityCenters_nearest'].min(), flats['cityCenters_nearest'].max()

Видим, что как правило квартиры из объявлений в наших данных располагаются от центра на 14.5 - 15 километров, и на 12 - 12.5 километров (два наивысших пика на графике). Имеются квартиры очень близкие (181 метр) к центру и сильно удаленные (66 километров) от него, они составляют размах наших данных. График мультимодален - можем проследить 5-7 вполне отчетливых пиков. Также имеется скос вправо, содержащий 2-3 собственных пика. Как можно понять, график весьма необычный и отличается от предыдущих. Выбросы представляют из себя квартиры, расположенные в большом удалении от центра.

Теперь посмотрим на гистограмму, составленную по столбцу 'airports_nearest':

In [ ]:
fig = px.histogram(flats, x='airports_nearest', title='Гистограмма расстояния до ближайшего аэропорта', 
                   color_discrete_sequence=['indianred'])
fig.show()

In [ ]:
flats['airports_nearest'].min(), flats['airports_nearest'].max()

Самое часто встречающееся расстояние до аэропорта - от 18.5 до 19 километров. Кроме того, часто встречаются такие интервалы расстояний как 24.5-25, 12.5-22. Разброс данных - от 6 километров до 85. График снова мультимодален (даже при манипуляциях с количеством bins), значения трех самых высоких пиков были перечислены ранее. Возможно, такая неоднородность данных может объясняться разным характером населенных пунктов, представленных в данных. Предположим, от Петербурга и крупных городов Ленинградской области до аэропорта добираться очевидно ближе, чем от отдаленных поселков, деревень и садовых товариществ, которые представлены в данных в большом количестве, создавая свои уплотнения и пики на графике. 

Наконец, взглянем на последнюю из гистограмм, описывающих отношения расстояния. А именно - расстояния до ближайшего парка, для чего нам понадобится столбец 'parks_nearest': 

In [ ]:
fig = px.histogram(flats, x='parks_nearest', title='Гистограмма расстояния до ближайшего парка', 
                   nbins=100, color_discrete_sequence=['indianred'])
fig.show()

In [ ]:
flats['parks_nearest'].min(), flats['parks_nearest'].max()

Как правило, до ближайшего парка от квартиры добираться 350-500 метров, а само расстояние в данных растягивается от 1 до 3190 метров. На графике представлено нормальное распределение с длинным правым хвостом, на котором расположились расстояния, большие 800 метров. Отметим, что этот график едва ли не больше всех остальных напоминает график нормального распределения: отсутствует сильный и резкий скос, наличествует один очевидный пик, относительно которого график (не считая правого хвоста) симметричен. 

И, напоследок, построим две гистограммы, говорящие нам о временном признаке наших данных - о дне ('weekday_exposition') и месяце ('month_exposition') публикации объявления:

In [ ]:
fig = px.histogram(flats, x='weekday_exposition', title='Гистограмма дня недели публикации объявления', 
                   color_discrete_sequence=['indianred'])
fig.show()

При взгляде на график сразу бросаетсся в глаза спад количества объявлений, опубликованных в выходные. Чаще же всего их публикуют в четверг и вторник. Вероятно, выходные рассматриваются людьми как отдых в том числе и от проведения операций с недвижимостью, для него выбирается начало и конец рабочей недели. Из будних же дней самый нечастный для формирования объявления о продаже квартиры - понедельник. 

In [ ]:
fig = px.histogram(flats, x='month_exposition', title='Гистограмма месяца публикации объявления', 
                   color_discrete_sequence=['indianred'])
fig.show()

В этом же случае очевидный пик для выставления квартиры на продажу - последний зимний (февраль) и два первых весенних (март и апрель) месяца. Меньше же всего это происходит в декабре, январе и мае - напрашивается предположение о том, что это связано с особой "праздничностью" этих месяцев. Летом публикационная активность поддерживается на среднем уровне, осенью постепенно поднимаясь на высокий (сравнимый с апрельским) уровень, зафиксированный в ноябре.

### 2. Изучение скорости продажи квартир 

Построим гистограмму по столбцу 'days_exposition' и по нему же найдем среднее арифметическое и медиану:

In [ ]:
fig = px.histogram(flats, x='days_exposition', title='Гистограмма скорости продаж', 
                   nbins=150, color_discrete_sequence=['indianred'])
fig.show()

In [ ]:
flats['days_exposition'].min(), flats['days_exposition'].max()

In [ ]:
flats['days_exposition'].mean()#Среднее арифметическое

In [ ]:
flats['days_exposition'].median()#Медиана

Как это следует из графика, чаще всего квартира продается за 30-50 дней, при том, что медианное значение равно 95 дням. Рекордно быстрая продажа заняла 1 день, долгая - больше 4 лет (1580 дней). Распределение нормальное с ощутимым положительным коэффициентом асимметрии и значительным количеством долгих сроков продаж, влияние которых можем видеть по среднему арифметическому, которое они "подняли" почти в 2 раза выше медианы (которая устойчива к такого рода выбросам). Продажи дольше 200 дней уже достаточно редки, после 600 дней становясь еще реже. 

### 3. Поик влияния некоторых факторов на цену квартиры

Построим диаграммы рассеяния для цены квартиры и некоторых выбранных параметров с целью проследить возможную взаимосвязь и влияние этих параметров на цену. Для аппроксимации точечных значений нашего графика воспользуемся обычным методом наименьших квадратов (ordinary least squares). Кроме того, помимо графика будем выводить и коэффициент корреляции Пирсона.

Как это будет видно, на графиках могут наблюдаться области уплотнения точек, что может затруднять зрительный анализ. Решение оставить графики в таком виде обосновано следующими соображениями - во-первых, вспомогательное использование аргумента trendline со значением 'ols' поможет упростить понимание "направленности" данных, во-вторых, вывод коэффициента корреляции также помогает удостовериться в правильности зрительного анализа, в-третьих, средства библиотеки Plotly позволяют вручную приблизить интересующий регион графика (инструмент 'Zoom'), к примеру, в районе скученности точек, и исследовать его более детально, наконец, в-четвертых - неудовлетворительность использования метода plot() библиотеки pandas, примеры которого будут даны ниже. Заметим, что манипуляции с аргументами метода никак ситуацию не улучшают (это заинтересованный читатель может проверить самостоятельно).

Первой характеристикой, влияние которой на цену ('last_price') мы проследим, будет общая площадь квартиры - 'total_area'.

In [ ]:
fig = px.scatter(flats, x='total_area', y='last_price', trendline='ols', 
                 title='Влияние общей площади на стоимость объекта')
fig.show()

In [ ]:
flats['total_area'].corr(flats['last_price'])

Как видим, график демонстрирует умеренную положительную связь между площадью квартиры и ее ценой - зависимость, о наличии которой свидетельствует также здравый смысл и опыт взаимодействия с рынком недвижимости (простая идея - чем квартира больше - тем она дороже).

Ниже приведем графики тех же двух переменных, созданные методом plot(). В первом из них область сильного уплотнения сохраняется независимо от значения аргумента alpha, при том он лишается преимуществ графика, построенного с помощью plotly (отсутствует возможность приближения/отдаления частей графика, при наведении на его точки мы не видим о них никакой информации). Во втором же сразу бросается в глаза его низкая информативность, независящая от значения аргумента gridsize - график предоставляет для анализа слишком мало шестиугольных ячеек. 

In [ ]:
flats.plot(x='total_area', y='last_price', kind='scatter', alpha=0.15, figsize=(15,10))

In [ ]:
flats.plot(x='total_area', y='last_price', kind='hexbin', gridsize=35, sharex=False, grid=True, figsize=(15,10))


Итак, будем и дальше пользоваться для построения диаграмм рассеяния средствами библиотеки Plotly. На этот раз изучим влияние на цену жилой площади ('living_area')

In [ ]:
fig = px.scatter(flats, x='living_area', y='last_price', trendline='ols', 
                 title='Влияние жилой площади на стоимость объекта')
fig.show()

In [ ]:
flats['living_area'].corr(flats['last_price'])

Сразу скажем, что положительная зависимость в этом случае и в случае с параметром площади кухни ожидаемым образом напоминает случай с общей площадью, действует правило "если больше общая площадь квартиры, то и ее жилая и кухонная площади будут больше" (при том, что кухонная площадь чаще всего увеличивается медленнее жилой), которое подтвердим, запросив соответствующие корреляции:

In [ ]:
flats['living_area'].corr(flats['total_area']), flats['kitchen_area'].corr(flats['total_area'])

Теперь взглянем и на зависимость цены от площади кухни ('kitchen_area'):

In [ ]:
fig = px.scatter(flats, x='kitchen_area', y='last_price', trendline='ols', 
                 title='Влияние кухонной площади на стоимость объекта')
fig.show()

In [ ]:
flats['kitchen_area'].corr(flats['last_price'])

Заметим два момента, характерных для всех трех случаев исследования зависимости цены от площади той или иной части объекта.

Во-первых, выбросы на этих трех графиках означают собой те квартиры, площади которых (общей, жилой или кухонной) соответствует нехарактерно высокая цена. 

Во-вторых, обратим внимание на интересную структуру расположения точек в зонах скученности, для примера которой используем приближенный фрагмент последнего графика:

In [ ]:
fig.update_xaxes(range=[9.9,10.6])
fig.update_yaxes(range=[0,15000000])

Видим, что точки плотно располагаются друг над другом в абсциссах, значение которых соответствует значению площади кухни. Действительно, площадь кухни скорее примет значение 10 или 10.2, чем 10.17 или 10.02. Это можно объяснить округлением значений при замере площади квартиры до более удобных и простых для восприятия.

Теперь же исследуем влияние на цену количества комнат 'rooms':

In [ ]:
fig = px.scatter(flats, x='rooms', y='last_price', trendline='ols', 
                 title='Влияние количества комнат на стоимость объекта')
fig.show()

In [ ]:
flats['rooms'].corr(flats['last_price'])

Зависимость на этом графике не так очевидна, если не пользоваться приближенным значением и коэффициентом корреляции. Ситуация становится проще, если с помощью приближения отдельных фрагментов заметить, что в вертикальных скоплениях точек сильнее всего уплотнение наблюдается к низу "столбцов", что частично видно и на общем плане графика. Заключим, что зависимость цены от количества комнат весьма мала. Пока что видим, что площадь (причем любая) влияет на цену ощутимо сильнее.

А сейчас рассмотрим влияния типа этажа ('floor_type') на цену всего объекта:

In [ ]:
fig = px.scatter(flats, x='floor_type', y='last_price', 
                 title='Влияние типа этажа на стоимость объекта')
fig.show()

Как видим, в этом случае провести анализ по графику будет довольно сложно. Попробуем воспользоваться другими способами:

In [ ]:
price_floor_data = flats.pivot_table(values='last_price', index='floor_type', aggfunc=['median', 'mean', 'count', 'min', 'max'])
price_floor_data.columns = ['price_median', 'price_mean', 'price_count', 'price_min', 'price_max']
price_floor_data

Видим, что, как правило, дороже всего стоят квартиры между первым и последним, следом идут квартиры, находящиеся на последнем этаже дома, а самые дешевые - те, что располагаются на первом. 

Попробуем еще один способ проследить зависимость. Определим функцию categorize_floor_numeric, с помощью которой сопоставим первому этажу 0, последнему - 1, а промежуточным этажам - их отношение с общим количеством этажей (таким образом, они займут место между 0 и 1 настолько близко к 1, насколько они близки к последнему этажу). Создадим столбец 'floor_type_numeric', в которой расположим результаты применения функции к строкам нашего датафрейма.<br>
(Вместо отношения конкретного этажа к максимальному могли просто возвращать 0.5, что на коэффициент корреляции сильно не повлияло бы, что легко может проверить интересующийся читатель)

In [ ]:
def categorize_floor_numeric(floor, top_floor):
    if floor == 1:
        return 0
    elif floor == top_floor:
        return 1
    else:
        return floor / top_floor

In [ ]:
flats['floor_type_numeric'] = flats.apply(lambda x: categorize_floor_numeric(x.floor, x.floors_total), axis=1)

Теперь взглянем на зависимости уже нового столбца и цены:

In [ ]:
fig = px.scatter(flats, x='floor_type_numeric', y='last_price', trendline='ols', 
                 title='Влияние типа этажа (выраженного на интервале [0,1]) на стоимость объекта')
fig.show()

In [ ]:
flats['floor_type_numeric'].corr(flats['last_price'])

Что же, теперь можем с уверенностью судить об отсутствии взаимосвязи между этими двумя параметрами.

Теперь проверим влияние даты размещения на цену объекта, будем использовать информацию о дне недели, месяце и годе размещения объявления. Кроме того, составим еще таблицы с такими параметрами как медиана, среднее, число записей, минимум и максимум. 

In [ ]:
fig = px.scatter(flats, x='weekday_exposition', y='last_price', trendline='ols', 
                 title='Влияние дня недели размещения объявления на стоимость объекта')
fig.show()

In [ ]:
fig = px.scatter(flats, x='month_exposition', y='last_price', trendline='ols', 
                 title='Влияние месяца размещения объявления на стоимость объекта')
fig.show()

In [ ]:
fig = px.scatter(flats, x='year_exposition', y='last_price', trendline='ols', 
                 title='Влияние года размещения объявления на стоимость объекта')
fig.show()

Во всех случаях можем видеть отсутствие влияния даты размещения объявления на стоимость объекта недвижимости. 

### 4. Расчет средней цены квадратного метра в топ-10 населенных пунктов по количеству объявлений

Создадим таблицу с информацией о 10 самых популярных среди продавцов населенных пунктах, в ней отобразим количество объявлений о квартирах, находящихся в этой локации и медианную и среднюю цену за квадратный метр в этой локации:

In [ ]:
localities_price_per_meter = (flats
                                   .pivot_table(values='price_per_meter', index='locality_name', 
                                                aggfunc=['count', 'median', 'mean']))

localities_price_per_meter.columns = ['count', 'median_price_per_meter', 'mean_price_per_meter']

top_localities = (localities_price_per_meter
                  .sort_values(by='count', ascending=False)
                  .head(10))
top_localities

Как можем видеть, дороже всего метр жилой недвижимости стоит в Санкт-Петербурге, а дешевле всего - в Выборге. 

### 5.  Определение цены каждого километра до центра Санкт-Петербурга

Итак, посмотрим, какова средняя цена для каждого километра удаления от центра города:

In [ ]:
spb_price_per_km = flats.pivot_table(values='last_price', index='cityCenters_nearest_km')
spb_price_per_km.columns = ['mean_price']
spb_price_per_km

Проанализируем получившуюся таблицу с помощью графика:

In [ ]:
fig = px.line(spb_price_per_km, x=spb_price_per_km.index, y='mean_price', markers=True)
fig.show()

На графике видим стремительное падение средней цены квартиры при увеличении расстояния от центра с 0 до 9 километров (с 31.4 до 7 миллионов). Затем при расстоянии от центра с 9 до 42 километров цена постепенно снижается с 7 до 3.1 миллионов, внезапно прыгая до 11.5 миллионов на 43 километре, и снова снижаясь к 5.1 миллионам на 45 километре, где идет постепенное снижение до 3.4 миллионов на 52 километре. Затем идет стремительный рост до 9 миллионов на 55 километре с еще более стремительным падением до 3 миллионов на 57 километре, после чего график ведет себя несколько более спокойно, вырастая до 4 миллионов на 66 километре с локальным пиком в 4,3 миллиона на 59 километре.

## 5. Общий вывод

В ходе исследования придерживались следующей траектории работы:
1. Предобработка загруженных данных<br>
На этом шаге мы исправляли некоторые недостатки в имеющихся данных. *Изучение пропусков* позволило разобраться в логике составления датафрейма, были выдвинуты некоторые гипотезы, объясняющие наличие пропусков в нескольких столбцах, в соответствие с этими гипотезами пропуски были заполнены. В тех же ячейках, где предложить логичный вариант ликвидации пропусков не удалось, - пропущенные значение оставили на месте. После этого была проведена *работа с типами данных* - были предложены более подходящие типы данных для некоторых столбцов, кроме того было пояснено ограничение, мешающее изменить тип данных в нескольких числовых столбцах на более удобный. Кроме того, важной частью этого этапа работы было *избавление от неявных дубликатов*, с помощью которого было уточнено несколько десятков названий населенных пунктов. Завершила этап предобработки *обработка аномальных значений*, где, снова, с помощью выдвижения некоторых гипотез и внедрения в логику составления датафрейма были уточнены выбивающиеся из общего ряда, нереалистичные и кажущиеся автору подозрительными значения.<br>
<br>
2. Добавление в таблицу новых столбцов<br>
Этот этап был непосредственной подготовкой к следующему - в наборе данных создали столбцы, выражающие характеристики объектов, которые были необходимы для проведения анализа.<br>
<br>
3. Проведение исследовательского анализа данных<br>
В течение этого этапа работы были проведены следующие аналитические действия:
    1. С помощью гистограмм проанализировали набор характеристик объектов<br>
    Этот шаг позволил нам сделать некоторые ценные замечания. Для начала, наши **данные обладают большим количеством выбросов**, как правило представляющих из себя уникальные квартиры - дорогие, большие по площади, с высокими потолками и большим количеством комнат. Значительное количество нестандартных значений объясняет тот факт, что **большая часть графиков демонстрирует положительный коэффициент асимметрии распределения**, что видно по скосу этих графиков в право, наличию на них длинного правого хвоста. Кроме того, **некоторые графики проявили свойство мультимодальности** по-разному объясняемые для каждого случая. Наконец, анализ позволил заметить, что **гистограммы демонстрируют интересные закономерности в дне недели и месяце публикации объявлений**.
    2. Был проведен анализ скорости продажи квартир<br>
    Анализ показал, что **чаще всего квартиры продаются в течение 2 месяцев**, при том, что **медианное значение срока продажи составляет 95 дней, а среднее арифметическое - 180**, что снова подчеркивает, что **влияние выбросов на среднее значение очень велико**. 
    3. Используя диаграммы рассеяния было проведено исследование о факторах, влияющих на цену квартиры<br>
    Выяснилось, что **сильнее всего прослеживается связь между ценой квартиры и ее площадью**, причем **на цену влияет как общая площадь квартиры, так и ее жилая и кухонная площади**. Здесь же выяснили, что **дата подачи объявления о продаже квартиры никак с ее ценой не коррелирует**.
    4. Был произведен расчет средней цены квадратного метра в городах-лидерах по количеству объявлений<br>
    В итоге было найдено, что **больше всего квадратный метр жилья стоит в Санкт-Петербурге, а меньше всего - в Выборге**, а **количество объявлений о продаже квартир в Петербурге превышает это количество в следующих за ним городах в 4.5 раза**. 
    5. С помощью линейного графика было исследовано влияние отдаленности квартиры от центра на его цену<br>
    Выявлено, что **высока скорость падения цен на жилье при удаления от центра на первые 3 километра**, а **с 9 километра цена движется гораздо плавнее**, несмотря на то, что **график демонстрирует неожиданные пики в точках 43 и 55 километров**.<br>

В качестве вывода, отсносящегося к методологии работы с данными, скажем, что **стало очевидным влияние выбросов, аномальных и нехарактерных для набора данных значений - именно они исказили нормальное распределение почти каждой характеристики**, кроме того, **была отчетливо прояснена важность применения тех или иных инструментов анализа данных, начиная от подбора библиотеки для визуализации, заканчивая выбором метода оценки характерного и наиболее частого для набора значения**, и, наконец, **выказала свою существенность необходимость наличия экспертных знаний в области, скажем, в вопросе определения соответствия рыночной норме той или иной высоты потолков, количества этажей в доме, балконов в квартире и т.д.**. 